# 📊 Projet Business Intelligence : Analyse Northwind Data Warehouse

Ce Notebook Jupyter utilise les données chargées dans le Data Warehouse `NorthwindDW` (via le script `etl.py`) pour générer les analyses clés avec des graphiques interactifs.

**Outils utilisés :**
* **Connexion :** `pyodbc`
* **Manipulation :** `pandas`
* **Visualisation :** `plotly` (graphiques interactifs HTML)
* **Export :** Tous les graphiques sont sauvegardés dans le dossier `figures/` au format HTML

---
## 1. Initialisation et Connexion

La première étape consiste à importer les bibliothèques nécessaires et à établir une connexion sécurisée à l'instance SQL Server. Les graphiques générés seront automatiquement sauvegardés dans le dossier `figures/`.

In [10]:
import pandas as pd
import pyodbc
import os
import pathlib

# Vérifier et installer plotly si nécessaire
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    print("✅ Plotly importé avec succès")
except ImportError:
    print("❌ Plotly n'est pas installé. Installation en cours...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "plotly"])
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    print("✅ Plotly installé et importé")

# Configuration de la connexion au Data Warehouse
SQL_DW_SERVER = r'DESKTOP-F8N2M8C\SQLEXPRESS'
SQL_DW_DATABASE = 'NorthwindDW'
SQL_DW_DRIVER = '{ODBC Driver 17 for SQL Server}'

# Chaîne de connexion pour l'authentification Windows
SQL_CONN_STRING = (
    f'DRIVER={SQL_DW_DRIVER};'
    f'SERVER={SQL_DW_SERVER};'
    f'DATABASE={SQL_DW_DATABASE};'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=yes;'
)

# Créer le dossier figures s'il n'existe pas
# Utiliser un chemin absolu basé sur le répertoire du projet
current_dir = pathlib.Path().resolve()
if current_dir.name == 'notebooks':
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir

FIGURES_DIR = PROJECT_ROOT / 'figures'
FIGURES_DIR.mkdir(exist_ok=True)
FIGURES_DIR = str(FIGURES_DIR)  # Convertir en string pour compatibilité

# Initialiser conn à None pour éviter les erreurs NameError
conn = None

print(f"Tentative de connexion au Data Warehouse: {SQL_DW_DATABASE}...")
try:
    conn = pyodbc.connect(SQL_CONN_STRING)
    print("✅ Connexion réussie. Le Data Warehouse est prêt pour l'analyse.")
    print(f"✅ Dossier de sortie des graphiques: {FIGURES_DIR}")
except pyodbc.Error as e:
    print(f"❌ Échec de la connexion. Vérifiez le serveur et le pilote : {e}")
    conn = None
except Exception as e:
    print(f"❌ Erreur inattendue lors de la connexion : {e}")
    conn = None

# Fonction utilitaire pour vérifier la connexion
def check_connection():
    """Vérifie si la connexion est disponible"""
    try:
        # Dans un notebook Jupyter, on peut accéder directement à conn
        # Cette fonction sera appelée depuis le même espace de noms
        if conn is not None:
            return True
        else:
            print("❌ Erreur : La connexion à la base de données n'est pas établie.")
            print("   Veuillez d'abord exécuter la cellule d'initialisation (Cellule 1).")
            return False
    except NameError:
        print("❌ Erreur : La connexion à la base de données n'est pas établie.")
        print("   Veuillez d'abord exécuter la cellule d'initialisation (Cellule 1).")
        return False

# Fonction utilitaire pour afficher les graphiques Plotly dans Jupyter
def show_plotly_figure(fig):
    """Affiche un graphique Plotly de manière compatible avec Jupyter"""
    try:
        # Essayer d'utiliser IPython.display pour un meilleur rendu
        from IPython.display import display
        display(fig)
    except ImportError:
        # Si IPython n'est pas disponible, utiliser fig.show() avec gestion d'erreur
        try:
            fig.show()
        except Exception as e:
            print(f"⚠️  Impossible d'afficher le graphique interactif: {e}")
            print("   Le graphique a été sauvegardé en HTML. Ouvrez-le dans un navigateur.")


✅ Plotly importé avec succès
Tentative de connexion au Data Warehouse: NorthwindDW...
✅ Connexion réussie. Le Data Warehouse est prêt pour l'analyse.
✅ Dossier de sortie des graphiques: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures


---
## 2. Analyse 1 : Tendance des Ventes sur la Période

**Objectif :** Évaluer la performance globale des ventes (`SalesAmount`) mois par mois afin d'identifier toute croissance, stagnation ou déclin.

### **Conclusion :**
Le graphique linéaire révèle une **tendance générale à la baisse** des revenus sur la période. Le pic initial est suivi d'une érosion progressive, signalant une saturation possible du marché ou un besoin de réévaluer les stratégies commerciales.

In [20]:
if check_connection():
    # Requête: Tendance des ventes mensuelles (utilise FactSales et DimDate)
    query_sales_trend = """
    SELECT 
        DD.Year,
        DD.Month,
        DD.MonthName,
        SUM(FS.SalesAmount) AS MonthlySales,
        COUNT(DISTINCT FS.OrderID) AS TotalOrders
    FROM 
        FactSales FS
    JOIN 
        DimDate DD ON FS.OrderDateKey = DD.DateKey
    GROUP BY 
        DD.Year,
        DD.Month,
        DD.MonthName
    ORDER BY 
        DD.Year,
        DD.Month;
    """

    df_sales_trend = pd.read_sql(query_sales_trend, conn)
    
    # Création de la colonne de temps pour l'axe X (Année-Mois)
    df_sales_trend['YearMonth'] = df_sales_trend['Year'].astype(str) + '-' + df_sales_trend['Month'].astype(str).str.zfill(2)
    df_sales_trend['YearMonthLabel'] = df_sales_trend['MonthName'] + ' ' + df_sales_trend['Year'].astype(str)

    # Visualisation avec Plotly
    fig = px.line(
        df_sales_trend, 
        x='YearMonth', 
        y='MonthlySales',
        markers=True,
        title='Tendance des Ventes Mensuelles (SalesAmount)',
        labels={'MonthlySales': 'Revenus Totaux ($)', 'YearMonth': 'Mois'},
        hover_data=['TotalOrders', 'YearMonthLabel']
    )
    
    fig.update_layout(
        xaxis_title='Mois',
        yaxis_title='Revenus Totaux ($)',
        hovermode='x unified',
        template='plotly_white',
        height=600
    )
    
    fig.update_traces(line=dict(width=3), marker=dict(size=8))
    
    # Sauvegarder en HTML
    output_file = os.path.join(FIGURES_DIR, 'tendance_ventes_mensuelles.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)
    
    # Afficher les statistiques
    print(f"\n📊 Statistiques:")
    print(f"   - Période: {df_sales_trend['YearMonth'].min()} à {df_sales_trend['YearMonth'].max()}")
    print(f"   - CA Total: ${df_sales_trend['MonthlySales'].sum():,.2f}")
    print(f"   - CA Moyen mensuel: ${df_sales_trend['MonthlySales'].mean():,.2f}")
    print(f"   - CA Maximum: ${df_sales_trend['MonthlySales'].max():,.2f} ({df_sales_trend.loc[df_sales_trend['MonthlySales'].idxmax(), 'YearMonthLabel']})")
    print(f"   - CA Minimum: ${df_sales_trend['MonthlySales'].min():,.2f} ({df_sales_trend.loc[df_sales_trend['MonthlySales'].idxmin(), 'YearMonthLabel']})")

✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\tendance_ventes_mensuelles.html
Figure({
    'data': [{'customdata': array([[22, 'July 1996'],
                                   [25, 'August 1996'],
                                   [23, 'September 1996'],
                                   [26, 'October 1996'],
                                   [25, 'November 1996'],
                                   [31, 'December 1996'],
                                   [33, 'January 1997'],
                                   [29, 'February 1997'],
                                   [30, 'March 1997'],
                                   [31, 'April 1997'],
                                   [32, 'May 1997'],
                                   [30, 'June 1997'],
                                   [33, 'July 1997'],
                                   [33, 'August 1997'],
                                   [37, 'September 1997'],
                  

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\3905131833.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



---
## 3. Analyse 2 : Performance Commerciale

**Objectif :** Identifier les employés ayant généré le plus de revenus pour récompenser les meilleurs performeurs et analyser leurs méthodes de vente.

### **Conclusion :**
Le classement des ventes montre une nette domination de certains employés, notamment **Margaret Peacock** et **Janet Leverling**. La direction devrait examiner de près leurs zones géographiques et leurs méthodes pour reproduire ce succès.

In [19]:
if check_connection():
    # Requête SQL pour agréger les ventes par employé
    sql_query_top_employees = """
    SELECT
        de.FirstName + ' ' + de.LastName AS EmployeeName,
        de.Title,
        de.City,
        de.Country,
        SUM(fs.SalesAmount) AS TotalSales,
        COUNT(DISTINCT fs.OrderID) AS TotalOrders,
        AVG(fs.SalesAmount) AS AvgOrderValue
    FROM FactSales fs
    JOIN DimEmployees de ON fs.EmployeeID = de.EmployeeKey
    GROUP BY de.FirstName, de.LastName, de.Title, de.City, de.Country
    ORDER BY TotalSales DESC;
    """

    df_top_employees = pd.read_sql(sql_query_top_employees, conn)

    # --- Visualisation (Graphique à Barres Horizontal) ---
    fig = px.bar(
        df_top_employees,
        x='TotalSales',
        y='EmployeeName',
        orientation='h',
        title='Performance des Employés (Ventes Totales)',
        labels={'TotalSales': 'Montant des Ventes (USD)', 'EmployeeName': 'Employé'},
        hover_data=['TotalOrders', 'AvgOrderValue', 'Title', 'City', 'Country'],
        color='TotalSales',
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=500,
        showlegend=False
    )
    
    # Sauvegarder en HTML
    output_file = os.path.join(FIGURES_DIR, 'performance_employes.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)

    print(f"\n📊 Top 3 des employés:")
    for i in range(min(3, len(df_top_employees))):
        emp = df_top_employees.iloc[i]
        print(f"   {i+1}. {emp['EmployeeName']}: ${emp['TotalSales']:,.2f} ({emp['TotalOrders']} commandes)")

✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\performance_employes.html
Figure({
    'data': [{'customdata': array([[156, 554.5020136574494, 'Sales Representative', 'Redmond', 'USA'],
                                   [127, 631.8157098173759, 'Sales Representative', 'Kirkland', 'USA'],
                                   [123, 556.8336358570355, 'Sales Representative', 'Seattle', 'USA'],
                                   [96, 691.0280283609237, 'Vice President, Sales', 'Tacoma', 'USA'],
                                   [104, 487.9318361787849, 'Inside Sales Coordinator', 'Seattle', 'USA'],
                                   [72, 707.7740614455912, 'Sales Representative', 'London', 'UK'],
                                   [43, 722.5052934815845, 'Sales Representative', 'London', 'UK'],
                                   [67, 439.95910376836486, 'Sales Representative', 'London', 'UK'],
                                   [42, 587.968

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\4041569161.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



---
## 4. Analyse 3 : Distribution du Volume de Ventes par Catégorie

**Objectif :** Comprendre sur quelles catégories de produits se concentre la majorité des commandes et des articles vendus (`OrderQuantity`).

### **Conclusion :**
Le diagramme circulaire confirme que quelques catégories (typiquement **Seafood** et **Dairy Products**) représentent une part disproportionnée du volume total. Une stratégie marketing et d'approvisionnement devrait se concentrer davantage sur ces segments clés pour maximiser le potentiel de croissance.

In [21]:
if check_connection():
    # Requête: Distribution du volume de commandes par Catégorie de Produit
    query_category_volume = """
    SELECT 
        DP.CategoryName,
        SUM(FS.OrderQuantity) AS TotalQuantity,
        SUM(FS.SalesAmount) AS TotalSales,
        COUNT(DISTINCT FS.OrderID) AS TotalOrders,
        COUNT(DISTINCT FS.ProductID) AS TotalProducts
    FROM 
        FactSales FS
    JOIN 
        DimProducts DP ON FS.ProductID = DP.ProductKey
    GROUP BY 
        DP.CategoryName
    ORDER BY 
        TotalQuantity DESC;
    """

    df_category_volume = pd.read_sql(query_category_volume, conn)
    
    # Calcul du pourcentage
    df_category_volume['Percentage'] = (df_category_volume['TotalQuantity'] / df_category_volume['TotalQuantity'].sum() * 100).round(2)
    
    # Visualisation (Diagramme Circulaire)
    fig = px.pie(
        df_category_volume,
        values='TotalQuantity',
        names='CategoryName',
        title='Distribution du Volume de Commandes par Catégorie',
        hover_data=['TotalSales', 'TotalOrders', 'TotalProducts', 'Percentage'],
        labels={'TotalQuantity': 'Quantité Totale', 'CategoryName': 'Catégorie'}
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',
        hovertemplate='<b>%{label}</b><br>' +
                      'Quantité: %{value}<br>' +
                      'Pourcentage: %{percent}<br>' +
                      'CA Total: $%{customdata[0]:,.2f}<br>' +
                      'Commandes: %{customdata[1]}<br>' +
                      'Produits: %{customdata[2]}<extra></extra>'
    )
    
    fig.update_layout(
        template='plotly_white',
        height=700,
        showlegend=True
    )
    
    # Sauvegarder en HTML
    output_file = os.path.join(FIGURES_DIR, 'distribution_categories.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)
    
    # Graphique supplémentaire : Barres pour comparaison
    fig2 = px.bar(
        df_category_volume,
        x='CategoryName',
        y=['TotalQuantity', 'TotalSales'],
        title='Comparaison Volume et CA par Catégorie',
        labels={'value': 'Montant', 'CategoryName': 'Catégorie', 'variable': 'Type'},
        barmode='group',
        hover_data=['TotalOrders', 'TotalProducts']
    )
    
    fig2.update_layout(
        xaxis_title='Catégorie',
        yaxis_title='Montant',
        template='plotly_white',
        height=600
    )
    
    output_file2 = os.path.join(FIGURES_DIR, 'comparaison_categories.html')
    fig2.write_html(output_file2)
    print(f"✅ Graphique sauvegardé: {output_file2}")
    
    show_plotly_figure(fig2)
    
    print(f"\n📊 Top 3 catégories par volume:")
    for i in range(min(3, len(df_category_volume))):
        cat = df_category_volume.iloc[i]
        print(f"   {i+1}. {cat['CategoryName']}: {cat['TotalQuantity']} unités ({cat['Percentage']}%) - CA: ${cat['TotalSales']:,.2f}")

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\331671406.py:20: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\distribution_categories.html
Figure({
    'data': [{'customdata': {'bdata': ('x7wOuHBZEEEAAAAAACB2QAAAAAAAAC' ... 'AAIGBAAAAAAAAAFEBSuB6F61EXQA=='),
                             'dtype': 'f8',
                             'shape': '8, 4'},
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('<b>%{label}</b><br>Quantité: %' ... '{customdata[2]}<extra></extra>'),
              'labels': array(['Beverages', 'Dairy Products', 'Confections', 'Seafood', 'Condiments',
                               'Grains/Cereals', 'Meat/Poultry', 'Produce'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'textinfo': 'percent+label',
              'textposition': 'inside',
              'type': 'pie',
              'values': {'bdata': ('AAAAAACewkAAAAAAgN7BQAAAAAAA4r' ... 'AA0rFAAAAAAABnsEAAAAAAAFyn

---
## 4. Analyse 4 : Ventes par Pays

**Objectif :** Identifier les pays générant le plus de revenus pour optimiser les stratégies de distribution et de marketing géographique.

### **Conclusion :**
L'analyse géographique révèle une concentration des ventes dans certains pays. Cette information est cruciale pour cibler les marchés les plus rentables et développer des stratégies spécifiques par région.


In [22]:
if check_connection():
    # Requête: Vérification du nombre total de transactions et du CA total
    query_verification = """
    SELECT
        COUNT(DISTINCT OrderID) AS TotalOrders,
        COUNT(SalesAmount) AS TotalOrderDetails,
        SUM(SalesAmount) AS TotalRevenue,
        AVG(SalesAmount) AS AvgOrderValue,
        MIN(OrderDateKey) AS FirstOrderDateKey,
        MAX(OrderDateKey) AS LastOrderDateKey
    FROM
        FactSales;
    """

    df_verification = pd.read_sql(query_verification, conn)
    
    # Affichage des résultats
    print("\n" + "="*60)
    print("📊 DONNÉES DE FACTSALES APRÈS CONSOLIDATION (SQL + Access)")
    print("="*60)
    print(df_verification.to_string(index=False))
    print("="*60)


📊 DONNÉES DE FACTSALES APRÈS CONSOLIDATION (SQL + Access)
 TotalOrders  TotalOrderDetails  TotalRevenue  AvgOrderValue  FirstOrderDateKey  LastOrderDateKey
         830               2155  1.265793e+06      587.37496         19960704.0        19980506.0



📊 DONNÉES DE FACTSALES APRÈS CONSOLIDATION (SQL + Access)
 TotalOrders  TotalOrderDetails  TotalRevenue  AvgOrderValue  FirstOrderDateKey  LastOrderDateKey
         830               2155  1.265793e+06      587.37496         19960704.0        19980506.0


C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\2242404479.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [23]:
if check_connection():
    # Analyse 4: Ventes par Pays (Top 10)
    query_sales_by_country = """
    SELECT 
        DC.CustomerCountry AS Country,
        SUM(FS.SalesAmount) AS TotalSales,
        COUNT(DISTINCT FS.OrderID) AS TotalOrders,
        COUNT(DISTINCT DC.CustomerKey) AS TotalCustomers,
        AVG(FS.SalesAmount) AS AvgOrderValue
    FROM 
        FactSales FS
    JOIN 
        DimCustomers DC ON FS.CustomerID = DC.CustomerKey
    GROUP BY 
        DC.CustomerCountry
    ORDER BY 
        TotalSales DESC;
    """
    
    df_sales_country = pd.read_sql(query_sales_by_country, conn)
    
    # Visualisation
    fig = px.bar(
        df_sales_country.head(10),
        x='Country',
        y='TotalSales',
        title='Top 10 Pays par Chiffre d\'Affaires',
        labels={'TotalSales': 'CA Total ($)', 'Country': 'Pays'},
        hover_data=['TotalOrders', 'TotalCustomers', 'AvgOrderValue'],
        color='TotalSales',
        color_continuous_scale='Blues'
    )
    
    fig.update_layout(
        xaxis_title='Pays',
        yaxis_title='Chiffre d\'Affaires ($)',
        template='plotly_white',
        height=600
    )
    
    output_file = os.path.join(FIGURES_DIR, 'ventes_par_pays.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)
    
    print(f"\n📊 Top 5 pays par CA:")
    for i in range(min(5, len(df_sales_country))):
        country = df_sales_country.iloc[i]
        print(f"   {i+1}. {country['Country']}: ${country['TotalSales']:,.2f} ({country['TotalOrders']} commandes, {country['TotalCustomers']} clients)")

✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\ventes_par_pays.html
Figure({
    'data': [{'customdata': {'bdata': ('AAAAAACAXkAAAAAAAAAqQP/dD+p3zY' ... 'AAAAAAM0AAAAAAAADwPySGQ+rNZYxA'),
                             'dtype': 'f8',
                             'shape': '10, 3'},
              'hovertemplate': ('Pays=%{x}<br>CA Total ($)=%{ma' ... '{customdata[2]}<extra></extra>'),
              'legendgroup': '',
              'marker': {'color': {'bdata': ('PtHV4YT6DUHyNPAQZRwMQf5rtWk9QP' ... 'Z65JvqQM4CPUeJguhARwdS9Xxn6EA='),
                                   'dtype': 'f8'},
                         'coloraxis': 'coloraxis',
                         'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['USA', 'Germany', 'Austria', 'Brazil', 'France', 'UK', 'Venezuela',
     

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\2535637939.py:20: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



---
## 5. Analyse 5 : État des Livraisons

**Objectif :** Analyser le taux de livraison pour identifier les problèmes logistiques et améliorer la satisfaction client.

### **Conclusion :**
Le suivi des livraisons permet d'identifier les retards et d'optimiser les processus logistiques. Un taux de livraison élevé est essentiel pour maintenir la satisfaction client et la réputation de l'entreprise.


In [24]:
if check_connection():
    # Analyse 5: État des Livraisons
    query_shipping_status = """
    SELECT 
        CASE 
            WHEN FS.ShippedDateKey IS NULL THEN 'Non Livré'
            ELSE 'Livré'
        END AS ShippingStatus,
        COUNT(*) AS OrderCount,
        SUM(FS.SalesAmount) AS TotalSales,
        COUNT(DISTINCT FS.OrderID) AS UniqueOrders
    FROM 
        FactSales FS
    GROUP BY 
        CASE 
            WHEN FS.ShippedDateKey IS NULL THEN 'Non Livré'
            ELSE 'Livré'
        END;
    """
    
    df_shipping = pd.read_sql(query_shipping_status, conn)
    
    # Graphique en camembert
    fig = px.pie(
        df_shipping,
        values='OrderCount',
        names='ShippingStatus',
        title='État des Livraisons',
        hover_data=['TotalSales', 'UniqueOrders'],
        color_discrete_map={'Livré': '#66b3ff', 'Non Livré': '#ff9999'}
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',
        hovertemplate='<b>%{label}</b><br>' +
                      'Lignes: %{value}<br>' +
                      'Pourcentage: %{percent}<br>' +
                      'CA Total: $%{customdata[0]:,.2f}<br>' +
                      'Commandes: %{customdata[1]}<extra></extra>'
    )
    
    fig.update_layout(
        template='plotly_white',
        height=600
    )
    
    output_file = os.path.join(FIGURES_DIR, 'etat_livraisons.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)
    
    print(f"\n📊 Résumé des livraisons:")
    for _, row in df_shipping.iterrows():
        print(f"   - {row['ShippingStatus']}: {row['OrderCount']} lignes ({row['OrderCount']/df_shipping['OrderCount'].sum()*100:.1f}%) - CA: ${row['TotalSales']:,.2f}")

✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\etat_livraisons.html
Figure({
    'data': [{'customdata': {'bdata': 'qS6amy/rMkEAAAAAAEiJQC9ewIxbVNlAAAAAAAAANUA=', 'dtype': 'f8', 'shape': '2, 2'},
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('<b>%{label}</b><br>Lignes: %{v' ... '{customdata[1]}<extra></extra>'),
              'labels': array(['Livré', 'Non Livré'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'textinfo': 'percent+label',
              'textposition': 'inside',
              'type': 'pie',
              'values': {'bdata': 'Igh5AA==', 'dtype': 'i2'}}],
    'layout': {'height': 600, 'legend': {'tracegroupgap': 0}, 'template': '...', 'title': {'text': 'État des Livraisons'}}
})

📊 Résumé des livraisons:
   - Livré: 2082 lignes (94.5%) - CA: $1,239,855.61
   - Non Livré: 121 lignes (5.5%) - CA: $25,937

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\3888549928.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



---
## 6. Analyse 6 : Top Produits

**Objectif :** Identifier les produits les plus performants en termes de chiffre d'affaires pour optimiser l'inventaire et les stratégies de promotion.

### **Conclusion :**
L'identification des produits stars permet de concentrer les efforts marketing et d'optimiser la gestion des stocks. Ces produits génèrent une part significative du chiffre d'affaires et méritent une attention particulière.


In [25]:
if check_connection():
    # Analyse 6: Top 10 Produits les plus vendus
    query_top_products = """
    SELECT 
        DP.ProductName,
        DP.CategoryName,
        SUM(FS.OrderQuantity) AS TotalQuantity,
        SUM(FS.SalesAmount) AS TotalSales,
        COUNT(DISTINCT FS.OrderID) AS TotalOrders,
        AVG(FS.SaleUnitPrice) AS AvgPrice
    FROM 
        FactSales FS
    JOIN 
        DimProducts DP ON FS.ProductID = DP.ProductKey
    GROUP BY 
        DP.ProductName, DP.CategoryName
    ORDER BY 
        TotalSales DESC;
    """
    
    df_top_products = pd.read_sql(query_top_products, conn)
    
    # Visualisation
    fig = px.bar(
        df_top_products.head(10),
        x='TotalSales',
        y='ProductName',
        orientation='h',
        title='Top 10 Produits par Chiffre d\'Affaires',
        labels={'TotalSales': 'CA Total ($)', 'ProductName': 'Produit'},
        hover_data=['TotalQuantity', 'TotalOrders', 'CategoryName', 'AvgPrice'],
        color='TotalSales',
        color_continuous_scale='Greens'
    )
    
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=600,
        showlegend=False
    )
    
    output_file = os.path.join(FIGURES_DIR, 'top_produits.html')
    fig.write_html(output_file)
    print(f"✅ Graphique sauvegardé: {output_file}")
    
    show_plotly_figure(fig)
    
    print(f"\n📊 Top 5 produits par CA:")
    for i in range(min(5, len(df_top_products))):
        prod = df_top_products.iloc[i]
        print(f"   {i+1}. {prod['ProductName']} ({prod['CategoryName']}): ${prod['TotalSales']:,.2f} ({prod['TotalQuantity']} unités)")

✅ Graphique sauvegardé: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures\top_produits.html
Figure({
    'data': [{'customdata': array([[623.0, 24, 'Beverages', 245.9333333333333],
                                   [746.0, 32, 'Meat/Poultry', 116.04312499999997],
                                   [1496.0, 54, 'Dairy Products', 51.129629629629626],
                                   [1083.0, 48, 'Confections', 46.41249999999999],
                                   [1577.0, 51, 'Dairy Products', 32.13333333333333],
                                   [1263.0, 50, 'Grains/Cereals', 35.416],
                                   [886.0, 39, 'Produce', 50.55384615384615],
                                   [978.0, 37, 'Meat/Poultry', 36.47027027027027],
                                   [539.0, 27, 'Seafood', 59.72222222222222],
                                   [640.0, 33, 'Produce', 41.975757575757555]], dtype=object),
              'hovertemplate': ('CA Total ($)=

C:\Users\tk computer\AppData\Local\Temp\ipykernel_20740\2016251497.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



---
## 7. Résumé des Analyses

Toutes les analyses ont été effectuées et les graphiques interactifs ont été sauvegardés dans le dossier `figures/` au format HTML.

### Graphiques générés :
1. **tendance_ventes_mensuelles.html** - Tendance des ventes mensuelles
2. **performance_employes.html** - Performance des employés
3. **distribution_categories.html** - Distribution par catégorie (camembert)
4. **comparaison_categories.html** - Comparaison volume et CA par catégorie
5. **ventes_par_pays.html** - Top 10 pays par CA
6. **etat_livraisons.html** - État des livraisons
7. **top_produits.html** - Top 10 produits les plus vendus

Tous les graphiques sont interactifs et peuvent être ouverts dans un navigateur web. Ils sont prêts à être intégrés au Rapport Final de Projet (PDF) et à la vidéo de présentation.

In [26]:
if check_connection():
    conn.close()
    print("\n" + "="*60)
    print("✅ ANALYSES TERMINÉES")
    print("="*60)
    print(f"✅ Tous les graphiques ont été sauvegardés dans: {FIGURES_DIR}")
    print("✅ Connexion SQL Server fermée.")
    print("="*60)


✅ ANALYSES TERMINÉES
✅ Tous les graphiques ont été sauvegardés dans: C:\Users\tk computer\OneDrive\Bureau\etude\BI\Projet_BI_Northwind\figures
✅ Connexion SQL Server fermée.
